In [2]:
from sklearn import linear_model
from sklearn import preprocessing
from sklearn.model_selection import train_test_split 
import pandas as pd
import numpy as np



#### Table des matieres

1. [**Import Dataset**](#1.-Import-Dataset)
2. [**Algorithmes ML de base**](## 2. Algorithmes ML de base)
    1.  [Dataset Vehicules](#2.1-Dataset-Vehicules)
    2.  [Dataset Caracteristiques](#2.2-Dataset-Caracteristiques)
    3.  [Dataset Usager](#2.3-Dataset-Usager)
    4.  [Dataset Lieux](#2.4-Dataset-Lieux)
3. [**Supression des colonnes(variables)**](#3.-Supression-des-colonnes(variables))
4. [**Gestion des Nan**](#4.-Gestion-des-Nan)
    1. [Remplacement -1 par Nan](#4.-Gestion-des-Nan)
    2.  [Dataset Vehicules](#4.2-Dataset-Vehicules)
    3.  [Dataset Caracteristiques](#3.3-Dataset-Caracteristiques)
    4.  [Dataset Usager](#4.4-Dataset-Usager)
    5.  [Dataset Lieux](#4.4-Dataset-Lieux)
5. [**Creation de variables suppl. et regroupement de categories**](#5.-Creation-de-variables-suppl.-et-regroupement-de-categories)
    1.  [Dataset Vehicules](#5.1-Dataset-Vehicules)
    2.  [Dataset Caracteristiques](#5.2-Dataset-Caracteristiques)
    3.  [Dataset Usager](#5.3-Dataset-Usager)
    4.  [Dataset Lieux](#5.4-Dataset-Lieux)
6. [**Gestion des doublons**](#6.-Gestion-des-doublons)
7. [**Sauvegarde dataset finale**](#7.-Sauvegarde-dataset-finale)

### 1. Import Dataset

In [3]:
path = 'data/'

######
years = ['2018f', '2019', '2020',  '2021' ]
s_df = ["carcteristiques","lieux-","vehicules-","usagers-"]
dff = pd.DataFrame()
k=0
i=0
hows='inner'
######

for year in years:
    df_1 =   pd.read_csv(path + "caracteristiques-" + year + '.csv', delimiter=';')
    df_2  =  pd.read_csv(path + "lieux-" + year + '.csv', delimiter=';')
    df_3  =  pd.read_csv(path + "vehicules-" + year + '.csv', delimiter=';')
    df_4 =   pd.read_csv(path + "usagers-" + year + '.csv', delimiter=';')

    merged_df = pd.merge(df_1,      df_2,on=['Num_Acc'], how=hows)
    merged_df = pd.merge(merged_df, df_3,on=['Num_Acc'], how=hows)
    if year=='2018f':
        df = pd.merge(merged_df, df_4,on=['Num_Acc','num_veh'], how=hows)
    else:
        df = pd.merge(merged_df, df_4,on=['Num_Acc','num_veh','id_vehicule'], how=hows)

    dff = pd.concat([dff, df],ignore_index=True)
    k=k+df.shape[0]
    #print('year : ', year,df.shape)
    i=i+1   
print("Total dataset 2018-2021 : ",dff.shape)


C:\Users\Jerome\AppData\Local\Temp\ipykernel_20192\2487081710.py:14: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2  =  pd.read_csv(path + "lieux-" + year + '.csv', delimiter=';')


Total dataset 2018-2021 :  (497594, 57)


In [ ]:
# standardisation ?

In [ ]:
# gest_dummies ?

In [ ]:
# train set / test set

### 2. Algorithmes ML de classification supervisée de base

#### 2.1 Régression logistique

In [ ]:
from sklearn.linear_model import LogisticRegression 
clf = LogisticRegression (C=1)
clf.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import confusion_matrix

y_pred = clf.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
cm = pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite'])
cm

In [ ]:
clf.score(X_test, y_test)

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

probs = clf.predict_proba(X_test)
y_preds = np.where(probs[:,1]>0.4,1,0)
cm = pd.crosstab(y_test, y_preds, rownames=['Classe réelle'], colnames=['Classe prédite'])

fpr, tpr, seuils = roc_curve(y_test, probs[:,1], pos_label=1)
roc_auc = auc(fpr, tpr)

plt.plot(fpr, tpr, color='orange', lw=2, label='Modèle clf (auc = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Aléatoire (auc = 0.5)')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Taux faux positifs')
plt.ylabel('Taux vrais positifs')
plt.title('Courbe ROC')
plt.legend(loc="lower right")
plt.show();

#### SVM

In [ ]:
from sklearn import svm

clf = svm.SVC(gamma=0.01 , kernel='poly')
clf.fit(X_train_scaled, y_train)

In [ ]:
from sklearn.metrics import confusion_matrix

y_pred= clf.predict(X_test)
confusion_matrix(y_test, y_pred)
pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite'])

In [ ]:
parametres = { 
    'C': [0.1,1,10], 
    'kernel' : ['rbf', 'linear','poly'],
    'gamma' : [0.001, 0.1, 0.5]
}

from sklearn.model_selection import GridSearchCV
grid_clf = GridSearchCV(estimator=clf, param_grid=parametres)
grille = grid_clf.fit(X_train_scaled,y_train)

print(pd.DataFrame.from_dict(grille.cv_results_).loc[:,['params', 'mean_test_score']]) 
grid_clf.best_params_ 
from sklearn.metrics import confusion_matrix

y_pred= grid_clf.predict(X_test_scaled)
confusion_matrix(y_test, y_pred)
pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite'])

target=y

In [ ]:
from sklearn.model_selection import learning_curve
import matplotlib.pyplot as plt
%matplotlib inline 

train_sizes, train_scores, valid_scores = learning_curve(svm.SVC(kernel='linear', C= 1), data, target, train_sizes=[50, 80, 110, 140], cv=5)

plt.xlabel("Training examples")
plt.ylabel("Score")

train_sizes=[50, 70, 80, 100, 110, 118]

train_sizes, train_scores, test_scores = learning_curve(
    grid_clf, data, target, n_jobs=4, train_sizes=train_sizes)

train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)
plt.grid()

plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, alpha=0.1,
                 color="r")
plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std, alpha=0.1, color="g")
plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
         label="Training score")
plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
         label="Cross-validation score")

plt.legend(loc="best")


#### KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn=  KNeighborsClassifier(n_neighbors=7, p=2)
knn.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import confusion_matrix

y_pred= knn.predict(X_test)
confusion_matrix(y_test, y_pred)
pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite'])

ARBRE DE DECISION

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
%matplotlib inline
# from interactions import show_tree

In [ ]:
df_tree = pd.read_csv('I:\PYFILES\merged_data_2018_2021_for_model.csv')

In [ ]:
df_tree.head()

In [ ]:
# Pour éviter le plantage lors du fit : passage de objet à integer/float

# Remplacer les valeurs dans la colonne 'age_usagers'
df_tree['age_usagers'] = df_tree['age_usagers'].replace(['0-13', '14-17','18-24', '25-34','35-44','45-54', '55-64', '65-74','75-plus'], [0, 1, 2, 3, 4, 5, 6, 7, 8])

# Remplacer les valeurs dans la colonne 'periode'
df_tree['periode'] = df_tree['periode'].replace(['Matin', 'Après-midi','Soir', 'Nuit'], [0, 1, 2, 3])

In [ ]:
# Création de la cible et matrice des features
target = df_tree.grav                                       # Vecteur cible
data = df_tree.drop(columns=['Num_Acc', 'grav'], axis=1)    # Matrice des Features

In [ ]:
# Création des jeux d'apprentissage et de test
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=123)

In [ ]:
# Création d'une instance d'arbre de décision, puis entrainement
dt_clf = DecisionTreeClassifier(criterion ='entropy', max_depth=32, random_state=123)
dt_clf.fit(X_train, y_train)   # Ajustement

In [ ]:
# Prédiction sur les données de test
y_pred = dt_clf.predict(X_test)
# Matrice de confusion
pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite'])

In [ ]:
# Stockage dans l'array feats, des features (caratérisques) importantes (les plus déterminantes)
feats = {}
for feature, importance in zip(data.columns, dt_clf.feature_importances_):
    feats[feature] = importance 
    
# Transformation en fd, puis tri    
importances = pd.DataFrame.from_dict(feats, orient='index').rename(columns={0: 'Importance'})
importances.sort_values(by='Importance', ascending=False).head(10)

#### RandomForest

In [ ]:
clf = sklearn.ensemble.RandomForestClassifier(n_jobs=-1, random_state=321 )
clf.fit(X_train, y_train)



In [ ]:
from sklearn.metrics import confusion_matrix

y_pred = clf.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
cm = pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite'])
cm

In [ ]:
clf.score(X_test, y_test)

In [ ]:
y_probas = clf.predict_proba(X_test)
y_probas

In [ ]:
import matplotlib.pyplot as plt
import scikitplot as skplt

skplt.metrics.plot_cumulative_gain(y_true=y_test, y_probas=y_probas)
plt.show()

#### VotingClassifier et Stacking

In [ ]:
from sklearn.ensemble import VotingClassifier, StackingClassifier
from sklearn.model_selection import train_test_split, KFold, cross_validate
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd

In [ ]:
clf1 = KNeighborsClassifier(n_neighbors=3)
clf2 = RandomForestClassifier( random_state= 123)
clf3 = LogisticRegression(max_iter=1000 )

vclf =  VotingClassifier(estimators=[('clf1_knn', clf1), ('clf2_rfc', clf2), ('clf3_lg', clf3)], voting='hard')

cv3 = KFold(n_splits=3, random_state=111, shuffle=True)

for clf, label in zip([clf1, clf2, clf3, vclf], ['KNN', 'Random Forest', 'Logistic Regression', 'Voting Classifier']):
    scores = cross_validate(clf, X_train, y_train, cv=cv3, scoring=['accuracy','f1'])
    print("[%s]: \n Accuracy: %0.2f (+/- %0.2f)" % (label, scores['test_accuracy'].mean(), scores['test_accuracy'].std()),
          "F1 score: %0.2f (+/- %0.2f)" % (scores['test_f1'].mean(), scores['test_f1'].std()))

In [ ]:
sclf = StackingClassifier(estimators=[('knn', clf1), ('rf', clf2), ('lr', clf3)], final_estimator=clf3)

scores = cross_validate(sclf, X_train, y_train, cv=cv3, scoring=['accuracy', 'f1'])
    
print("[StackingClassifier]: \n Accuracy: %0.2f (+/- %0.2f)\n" % (scores['test_accuracy'].mean(), scores['test_accuracy'].std()),
      "F1 score: %0.2f (+/- %0.2f)" % (scores['test_f1'].mean(), scores['test_f1'].std()))

In [ ]:
vclf.fit(X_train, y_train)
sclf.fit(X_train, y_train)

print("Acc :", vclf.score(X_test, y_test))
print("Acc :", sclf.score(X_test, y_test))